# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from generative_social_choice.utils.helper_functions import get_base_dir_path

In [29]:
from generative_social_choice.slates.voting_algorithms import (
    SequentialPhragmenMinimax,
    GreedyTotalUtilityMaximization,
    ExactTotalUtilityMaximization,
    LPTotalUtilityMaximization,
    VotingAlgorithm,
    GeometricTransformation,
)
from generative_social_choice.slates.voting_algorithm_axioms import (
    IndividualParetoAxiom,
    HappiestParetoAxiom,
    CoverageAxiom,
    MinimumAndTotalUtilityParetoAxiom,
    VotingAlgorithmAxiom,
    NonRadicalMinUtilityAxiom,
    NonRadicalTotalUtilityAxiom,
)
from generative_social_choice.test.utilities_for_testing import rated_vote_cases

# Load Data

In [4]:
voting_algorithm_evals_dir = get_base_dir_path() / "data" / "voting_algorithm_evals"
latest = True
if latest:
    file = sorted(voting_algorithm_evals_dir.glob("*.csv"))[-1]
else:
    file = voting_algorithm_evals_dir / "2025-01-20-180945.csv"
file

df = pd.read_csv(file, index_col=0, header=[0, 1])

In [5]:
file

PosixPath('/workspaces/generative_social_choice/generative_social_choice/data/voting_algorithm_evals/2025-11-08-010606.csv')

In [6]:
# Select subset of rows
# df = df.iloc[-8:]
df

vote,Simple 1,Simple 2,Simple 3,Ex 1.1,Ex 1.1 modified,Ex 1.2,Ex A.1,Ex 1.3,Ex 2.1,Ex 2.2,...,Ex C.1,Ex C.2,Ex D.1,Ex Alg1.3,Ex Alg1.4,Ex Alg1.5,Ex Alg2.1,Ex Alg A.1,Ex Alg A.2,Ex NRU1
subtest,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,...,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency
ExactTotalUtilityMaximization(utility_transform=None),1.0,1.0,1.0,1.0,0.333333,1.0,1.0,1.0,1.0,1.0,...,1.0,0.416667,1.0,1.0,1.0,1.0,0.791667,1.0,1.0,0.958333
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.000000


## Overall Performance

In [7]:
df.sum(axis=1)

ExactTotalUtilityMaximization(utility_transform=None)                              25.666667
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))    28.000000
dtype: float64

In [8]:
pd.DataFrame(np.trunc(df.values + 0/24+1*1e-6), index=df.index, columns=df.columns).sum(axis=1)

ExactTotalUtilityMaximization(utility_transform=None)                              21.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))    28.0
dtype: float64

## Differential Test Performance

In [9]:
algs_for_non_uniform_rows = df.loc[[ExactTotalUtilityMaximization().name, ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)).name], :]

non_uniform_columns = algs_for_non_uniform_rows.loc[:, algs_for_non_uniform_rows.nunique() > 1]
non_uniform_columns


vote,Ex 1.1 modified,Ex 3.1,Ex 4.1,Ex B.3,Ex C.2,Ex Alg2.1,Ex NRU1
subtest,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency
ExactTotalUtilityMaximization(utility_transform=None),0.333333,0.958333,0.583333,0.625,0.416667,0.791667,0.958333
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.000000,1.000000,1.000000,1.000,1.000000,1.000000,1.000000


These counts show which test cases and which axioms most frequently show unique behavior across the `VotingAlgorithm`s

In [10]:
# Count the number of columns with each name in both levels of the MultiIndex
level_0_counts = non_uniform_columns.columns.get_level_values(0).value_counts()
level_1_counts = non_uniform_columns.columns.get_level_values(1).value_counts()

print("Counts for level 0:")
print(level_0_counts)
print("\nCounts for level 1:")
print(level_1_counts)



Counts for level 0:
vote
Ex 1.1 modified    1
Ex 3.1             1
Ex 4.1             1
Ex B.3             1
Ex C.2             1
Ex Alg2.1          1
Ex NRU1            1
Name: count, dtype: int64

Counts for level 1:
subtest
Non-radical Total Utility Pareto Efficiency    7
Name: count, dtype: int64


## Axiom Pass Rate

In [11]:
df.groupby(df.columns.get_level_values(1), axis=1).mean()

/tmp/ipykernel_59682/1810388850.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df.groupby(df.columns.get_level_values(1), axis=1).mean()


subtest,Non-radical Total Utility Pareto Efficiency
ExactTotalUtilityMaximization(utility_transform=None),0.916667
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.000000


# Debuggign

In [12]:
# alg = GreedyTotalUtilityMaximization()
# alg = LPTotalUtilityMaximization()
# alg = SequentialPhragmenMinimax()
# alg = ExactTotalUtilityMaximization()
alg = ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = SequentialPhragmenMinimax(load_magnitude_method="marginal_slate", clear_reassigned_loads=False, redistribute_defected_candidate_loads=False)

In [13]:
df1 = df.loc[alg.name,:].unstack()
df1

subtest,Non-radical Total Utility Pareto Efficiency
vote,
Ex 1.1,1.0
Ex 1.1 modified,1.0
Ex 1.2,1.0
Ex 1.3,1.0
Ex 2.1,1.0
Ex 2.2,1.0
Ex 3.1,1.0
Ex 4.1,1.0
Ex 4.2,1.0


In [28]:
case = rated_vote_cases["Ex NRU1"]
case.rated_votes

,s1,s2
0,20.0,20.100
1,20.0,19.904


In [30]:
# case = rated_vote_cases["Ex 1.1 modified"]
from generative_social_choice.slates.voting_algorithms import RatedVoteCase
from generative_social_choice.slates.voting_utils import (
    CellWiseAugmentation, CandidateWiseAugmentation, VoterAndCellWiseAugmentation)


# case = rated_vote_cases["Ex A.1"]
case = rated_vote_cases["Ex NRU1"]

# case = RatedVoteCase(
#     name="Ex BIG",
#     rated_votes=case.rated_votes,
#     slate_size=case.slate_size,
#     noise_augmentation_methods = {
#                 CellWiseAugmentation(): 40, 
#                 CandidateWiseAugmentation(): 100, 
#                 VoterAndCellWiseAugmentation(): 100
#             }
# )
# case = rated_vote_cases["Ex 4.3"]
# case = rated_vote_cases["Simple 3"]
# axiom = CoverageAxiom()
# axiom = HappiestParetoAxiom()
# axiom = IndividualParetoAxiom()
# axiom = MinimumAndTotalUtilityParetoAxiom()
axiom = NonRadicalTotalUtilityAxiom()
# axiom = NonRadicalMinUtilityAxiom()

case.rated_votes.tail()

,s1,s2
0,20.0,20.100
1,20.0,19.904


In [115]:
alg.name

'ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))'

In [116]:
# df.loc[:,df.columns.get_level_values(1) == axiom.name].iloc[:,10:-10]
df.loc[alg.name,df.columns.get_level_values(1) == axiom.name]
# df.columns.get_level_values(1) == type(axiom).__name__
# df.columns.get_level_values(1), axiom.name


vote             subtest                                    
Simple 1         Non-radical Total Utility Pareto Efficiency    1.0
Simple 2         Non-radical Total Utility Pareto Efficiency    1.0
Simple 3         Non-radical Total Utility Pareto Efficiency    1.0
Ex 1.1           Non-radical Total Utility Pareto Efficiency    1.0
Ex 1.1 modified  Non-radical Total Utility Pareto Efficiency    1.0
Ex 1.2           Non-radical Total Utility Pareto Efficiency    1.0
Ex A.1           Non-radical Total Utility Pareto Efficiency    1.0
Ex 1.3           Non-radical Total Utility Pareto Efficiency    1.0
Ex 2.1           Non-radical Total Utility Pareto Efficiency    1.0
Ex 2.2           Non-radical Total Utility Pareto Efficiency    1.0
Ex 3.1           Non-radical Total Utility Pareto Efficiency    1.0
Ex 4.1           Non-radical Total Utility Pareto Efficiency    1.0
Ex 4.2           Non-radical Total Utility Pareto Efficiency    1.0
Ex 4.3           Non-radical Total Utility Pareto Effic

In [117]:
alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)[0]

['s1']

In [118]:
# axiom.satisfactory_slates(rated_votes=case.rated_votes, slate_size=case.slate_size)
# case.augmented_cases[0]

In [119]:
num_aug_cases = len(case.augmented_cases)
aug_case_votes = pd.DataFrame(index = range(num_aug_cases), columns=["rated_votes", "assignments", "axiom_slates", "alg_slate", "satisfied"])
for i, aug_case in enumerate(case.augmented_cases):
    # print(aug_case)
    aug_case_votes.at[i, "rated_votes"] = aug_case
    aug_case_votes.at[i, "axiom_slates"] = axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "alg_slate"], aug_case_votes.at[i, "assignments"] = alg.vote(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "satisfied"] = axiom.evaluate_assignment(
        rated_votes=aug_case,
        assignments=aug_case_votes.at[i, "assignments"],
        slate_size=case.slate_size,
    )
    # print(axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size))
# aug_case_votes.iloc[:,2:]

In [120]:
failures = aug_case_votes[aug_case_votes["satisfied"] == False]
failures.iloc[:,2:]


,axiom_slates,alg_slate,satisfied


In [121]:
f0 = failures.iloc[0]
f0.rated_votes
# (f0.rated_votes - case.rated_votes)*1e6


IndexError: single positional indexer is out-of-bounds

In [ ]:
((f0.rated_votes - case.rated_votes)*1e6)

,s1,s2
0,46469.419565,1261.748277
1,46469.419565,1261.748277
2,46469.419565,1261.748277
3,46469.419565,1261.748277
4,46469.419565,1261.748277
5,46469.419565,1261.748277
6,46469.419565,1261.748277
7,46469.419565,1261.748277
8,46469.419565,1261.748277
9,46469.419565,1261.748277


In [ ]:
f0.assignments

,candidate_id
0,s2
1,s2
2,s2
3,s2
4,s2
5,s2
6,s2
7,s2
8,s2
9,s2


In [ ]:
alg.vote(rated_votes=f0.rated_votes, slate_size=case.slate_size)

(['s2'],
    candidate_id
 0            s2
 1            s2
 2            s2
 3            s2
 4            s2
 5            s2
 6            s2
 7            s2
 8            s2
 9            s2
 10           s2)

In [ ]:
axiom.evaluate_assignment(
    rated_votes=f0.rated_votes,
    assignments=f0.assignments,
    slate_size=case.slate_size,
)

False

In [ ]:
from kiwiutils.kiwilib import leafClasses
[cls.__name__ for cls in leafClasses(VotingAlgorithmAxiom)]

['NonRadicalMinUtilityAxiom',
 'IndividualParetoAxiom',
 'MinimumAndTotalUtilityParetoAxiom',
 'CoverageAxiom',
 'HappiestParetoAxiom',
 'NonRadicalTotalUtilityAxiom']

# Scratch

In [36]:
alg1 = ExactTotalUtilityMaximization()
_, asg = alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)
asg

,candidate_id
0,s2
1,s2


In [35]:
axiom.evaluate_assignment(
    rated_votes=case.rated_votes,
    assignments=asg,
    slate_size=case.slate_size,
)

False